In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score
from plot_helpers import plot_interpolated
import os
import main
import keras
import masked_metrics
import models

2025-12-10 17:46:19.405956: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-10 17:46:19.406303: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-10 17:46:19.458431: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-10 17:46:22.415943: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
train_ds, test_ds, val_ds, Xs, Xc, Y, ds = main.construct_dataset(100)

E0000 00:00:1765406787.412881 1769922 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
E0000 00:00:1765406787.422278 1769922 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1765406787.424546 1769922 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def r2(Y_true:np.ndarray, Y_pred:np.ndarray, seq2seq:bool, context:bool, display:bool=True): 
    Y_true_sliced = Y_true[:,-1,:]
    Y_pred_sliced = Y_pred
    if seq2seq: 
        Y_pred_sliced = Y_pred[:,-1,:] 
    Y_cat = np.concat([Y_pred_sliced, Y_true_sliced], -1) 
    Y_cat = Y_cat[~np.isnan(Y_cat).any(axis=-1)] 
    Y_cat = np.stack([Y_cat[:,:5], Y_cat[:,5:]], axis=0)
    score = r2_score(Y_cat[1], Y_cat[0], multioutput='raw_values') 
    if display:
        print(
            f"seq2seq={seq2seq}, context={context}:\t{score}"
        )
    return score


In [ ]:
Y_inter_s_c     = np.load(os.path.normpath('runs/2025-12-08_20-28-11/y_inter.npz'))['arr_0']
Y_pred_s_c      = np.load(os.path.normpath('runs/2025-12-08_20-28-11/y_pred.npz'))['arr_0']
Y_inter_s_nc    = np.load(os.path.normpath('runs/2025-12-08_20-39-41/y_inter.npz'))['arr_0']
Y_pred_s_nc     = np.load(os.path.normpath('runs/2025-12-08_20-39-41/y_pred.npz'))['arr_0']
Y_inter_ns_c    = np.load(os.path.normpath('runs/2025-12-08_20-07-00/y_inter.npz'))['arr_0']
Y_pred_ns_c     = np.load(os.path.normpath('runs/2025-12-08_20-07-00/y_pred.npz'))['arr_0']
Y_inter_ns_nc   = np.load(os.path.normpath('runs/2025-12-08_20-54-44/y_inter.npz'))['arr_0']
Y_pred_ns_nc    = np.load(os.path.normpath('runs/2025-12-08_20-54-44/y_pred.npz'))['arr_0']

arg_list = [
    (Y_pred_s_c, True, True), 
    (Y_pred_s_nc, True, False), 
    (Y_pred_ns_c, False, True),
    (Y_pred_ns_nc, False, False)
]

Y_np = Y.numpy()
list(map(lambda T: r2(Y_np, *T), arg_list))

In [ ]:
Y_pred = np.load(os.path.normpath('runs/2025-12-10_17-29-54/y_pred.npz'))
r2(Y.numpy(), Y_pred, False, True)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)